In [111]:
import numpy as np
import pandas as pd
import os 
import sys
sys.path.append(os.path.abspath('../common')) # add path to common functions

In [112]:
from preprocess import getdfs

train_df, valid_df = getdfs(data = 'text_seq', train_size = 1)

In [113]:
train_df['input_str'] = train_df['input_str'].apply(lambda x : x[3:])
valid_df['input_str'] = valid_df['input_str'].apply(lambda x : x[3:])

In [114]:
num_feat = 47

def get_columns (df) :
    for i in range(num_feat):
        df[f'c_{i}'] = df['input_str'].apply(lambda x : x[i])
    return df.drop(columns = ['input_str'])

train_df = get_columns(train_df)
valid_df = get_columns(valid_df)

In [115]:
import pandas as pd

df = train_df

# Convert string values in the 47 columns to integers
df.iloc[:, :-1] = df.iloc[:, :-1].astype(int)  # Assuming last column is the target

# Initialize a DataFrame to store counts of each number (0-9)
count_df = pd.DataFrame(0, index=df.index, columns=[f'count_{i}' for i in range(10)])

# For each number (0-9), count its occurrences across the 47 columns
for i in range(10):
    count_df[f'count_{i}'] = (df.iloc[:, :-1] == i).sum(axis=1)

# Add the binary target column to the count_df for correlation calculation
count_df['label'] = df['label']  # Assuming 'target' is the name of the binary target column

# Calculate the Pearson correlation between the count of each number (0-9) and the target
correlations = count_df.corr()['label'].drop('label')

# Display the correlations
print(correlations)


count_0   -0.434806
count_1    0.378802
count_2    0.006309
count_3    0.002525
count_4   -0.016974
count_5   -0.020814
count_6   -0.000944
count_7    0.012974
count_8    0.027284
count_9   -0.000509
Name: label, dtype: float64


In [116]:
df

,label,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,...,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46
0,0,0,1,5,4,3,6,4,6,4,...,1,5,9,6,2,6,2,6,1,4
1,0,4,6,4,1,5,9,6,3,6,...,4,7,6,1,6,1,4,2,8,4
2,0,1,5,4,3,6,2,6,2,1,...,0,5,1,1,5,9,6,2,8,4
3,1,0,1,5,4,3,6,4,2,2,...,3,5,1,0,6,1,4,2,6,2
4,1,4,6,4,1,8,9,9,4,2,...,1,5,9,6,6,1,4,2,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,1,0,0,2,6,2,6,7,0,2,...,1,4,1,5,9,6,2,4,1,4
7076,1,0,4,6,4,4,2,2,1,5,...,6,1,4,3,9,0,1,2,6,2
7077,1,3,0,6,9,9,1,5,9,6,...,2,6,2,6,1,4,1,5,9,6
7078,0,0,6,1,4,1,5,9,6,1,...,6,9,4,0,6,1,4,2,8,4


In [117]:
from preprocess import getdfs

train_df, valid_df = getdfs(data = 'text_seq', train_size = 1)
# train_df['input_str'] = train_df['input_str'].apply(lambda x : x[3:])
# valid_df['input_str'] = valid_df['input_str'].apply(lambda x : x[3:])

In [118]:
# for index,row in train_df.iterrows() :
    # print(row['label'])
    # a = row['input_str']
    # i = 0
    # while a[i] == '0' :
    #     i += 1
    # print(a[i:])  
    # for i in range (8) :
    #     print(row[f'input_str'][4*i:4*(i+1)], end = ' ')
    # for i in range (8, 13) :
    #     print(row[f'input_str'][32 + 3*(i-8) : 32 + 3*(i-7)], end = ' ')
    # print()

In [119]:
from preprocess import getdfs

train_df, valid_df = getdfs(data = 'text_seq', train_size = 1)

In [120]:
import tensorflow as tf
from tensorflow.keras import layers

# Number of unique digits (0-9) in the input and emojis (13 unique emojis) in the output
num_input_tokens = 10  # digits 0-9
num_output_emojis = 13  # the number of unique emojis
embedding_dim = 32  # Embedding size for each digit
hidden_units = 16  # LSTM/GRU units

# Model architecture
def build_seq2seq_model(input_length=50, output_length=13):
    # Input Layer (50-length sequence of digits 0-9)
    encoder_inputs = layers.Input(shape=(input_length,))
    
    # Embedding layer (Converting digit inputs into dense vectors)
    x = layers.Embedding(input_dim=num_input_tokens, output_dim=embedding_dim, input_length=input_length)(encoder_inputs)
    
    # Encoder (LSTM or GRU to encode the sequence of digits)
    encoder_outputs, state_h, state_c = layers.LSTM(hidden_units, return_state=True)(x)
    
    # Repeat the encoder's context vector (state) for the number of output emojis (13)
    decoder_inputs = layers.RepeatVector(output_length)(encoder_outputs)
    
    # Decoder (Using an LSTM or GRU to generate the sequence of emojis)
    decoder_lstm = layers.LSTM(hidden_units, return_sequences=True)(decoder_inputs, initial_state=[state_h, state_c])
    
    # Output Layer (Predicting emojis using a softmax layer)
    decoder_outputs = layers.TimeDistributed(layers.Dense(num_output_emojis, activation='softmax'))(decoder_lstm)
    
    # Build the model
    model = tf.keras.Model(encoder_inputs, decoder_outputs)
    
    return model

# Build the model
seq2seq_model = build_seq2seq_model()

# Compile the model (assuming we're using categorical crossentropy for multi-class classification)
seq2seq_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
seq2seq_model.summary()


/opt/miniconda3/envs/cs771/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 50, 32)    │        320 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ [(None, 16),      │      3,136 │ embedding_3[0][0] │
│                     │ (None, 16),       │            │                   │
│                     │ (None, 16)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_3     │ (None, 13, 16)    │          0 │ lstm_6[0][0]      │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 13, 16)    │      2,112 │ repeat_vector_3[… │
│                     │                   │            │ lstm_6[0][1],     │
│                     │                   │            │ lstm_6[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 13, 13)    │        221 │ lstm_7[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,789 (22.61 KB)

 Trainable params: 5,789 (22.61 KB)

 Non-trainable params: 0 (0.00 B)

In [146]:
x_train = train_df.drop(columns = ['label'])
x_valid = valid_df.drop(columns = ['label'])

In [147]:
y_train, y_valid = pd.read_csv('../datasets/train/train_emoticon.csv'), pd.read_csv('../datasets/valid/valid_emoticon.csv')

In [148]:
y_train = y_train.drop(columns = ['label'])
y_valid = y_valid.drop(columns = ['label'])

In [124]:
seq2seq_model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=10, batch_size=32)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 1), output.shape=(None, 13, 13)

In [149]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

# Sample dataframes: (You would load your actual data here)
# Assume df_input contains a column 'digit_sequence' with 50-digit strings
# Assume df_target contains a column 'emoji_sequence' with corresponding emoji sequences
# Convert digits to tensor
def digit_to_tensor(digit_str):
    # Convert string of digits to a list of integers (0-9), then to a tensor
    return torch.tensor([int(digit) for digit in digit_str], dtype=torch.long)

# # Preprocess

In [150]:
emojis, _ = getdfs(data = 'emoticon', train_size = 1)
emojis.drop(columns = ['label'], inplace = True)
emoji_vocab = pd.unique(emojis.values.ravel())
emoji_vocab.shape

(214,)

In [151]:
def emoji_to_tensor(emoji_str):
    # Convert string of emojis to a tensor
    emoji_to_index = {emoji: idx for idx, emoji in enumerate(emoji_vocab)}
    return torch.tensor([emoji_to_index[emoji] for emoji in emoji_str], dtype=torch.long)

In [152]:

# Convert the digit sequences to tensors
X_train = torch.stack([digit_to_tensor(seq) for seq in x_train['input_str']])
Y_train = torch.stack([emoji_to_tensor(seq) for seq in y_train['input_emoticon']])
X_valid = torch.stack([digit_to_tensor(seq) for seq in x_valid['input_str']])
Y_valid = torch.stack([emoji_to_tensor(seq) for seq in y_valid['input_emoticon']])
# convert the list of tensors to torch tensor

In [129]:

# # Split into train and test sets
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Hyperparameters
input_dim = 10  # 10 possible digits (0-9)
output_dim = 216  # 13 possible emojis
embedding_dim = 16
hidden_dim = 128
seq_len = 50
emoji_seq_len = 13

### Simplified Model ###
class SimpleSeq2Seq(nn.Module):
    def __init__(self, input_dim, output_dim, embedding_dim, hidden_dim, seq_len, emoji_seq_len):
        super(SimpleSeq2Seq, self).__init__()
        
        # Embedding layer for digit sequence
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        # Fully connected layers (linear transformation)
        self.fc1 = nn.Linear(embedding_dim * seq_len, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim * emoji_seq_len)
        
    def forward(self, x):
        # Pass through embedding layer
        x = self.embedding(x)  # Shape: (batch_size, seq_len, embedding_dim)
        
        # Flatten the sequence of embeddings into a single vector
        x = x.view(x.size(0), -1)  # Shape: (batch_size, embedding_dim * seq_len)
        
        # Pass through fully connected layers
        x = torch.relu(self.fc1(x))  # Shape: (batch_size, hidden_dim)
        x = self.fc2(x)  # Shape: (batch_size, output_dim * emoji_seq_len)
        
        # Reshape the output to get emoji_seq_len sequences of output_dim emojis
        x = x.view(x.size(0), emoji_seq_len, output_dim)  # Shape: (batch_size, emoji_seq_len, output_dim)
        
        return x

# Instantiate the model
model = SimpleSeq2Seq(input_dim, output_dim, embedding_dim, hidden_dim, seq_len, emoji_seq_len)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training Loop ###
def train(model, X_train, Y_train, optimizer, criterion, epochs=100, batch_size=2):
    model.train()
    
    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, len(X_train), batch_size):
            # Get a batch
            x_batch = X_train[i:i + batch_size]
            y_batch = Y_train[i:i + batch_size]
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(x_batch)
            
            # Reshape for loss computation
            outputs = outputs.view(-1, output_dim)  # Shape: (batch_size * emoji_seq_len, output_dim)
            y_batch = y_batch.view(-1)  # Shape: (batch_size * emoji_seq_len)
            
            # Compute the loss
            loss = criterion(outputs, y_batch)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(X_train)}')

# Train the model
train(model, X_train, Y_train, optimizer, criterion, epochs=100)

### Evaluation ###
def evaluate(model, X_test, Y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        predictions = outputs.argmax(dim=2)  # Get the most likely emoji for each position
        accuracy = (predictions == Y_test).float().mean().item()
        print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Evaluate the model on test set
evaluate(model, X_valid, Y_valid)


Epoch 1, Loss: 1.2641056718630979
Epoch 2, Loss: 0.8049397812808974
Epoch 3, Loss: 0.5251180117244775


KeyboardInterrupt: 

find the 216 substrings that pop up most frequently.

In [141]:
x_train

0        1543646427181596614280026242231322841596262614
1       46415963695154364222626141104715962624761614284
2       15436262159659996144224641358061426240511596284
3        1543642246412386142621596895615962843510614262
4       46418994221543620690159661426247612621596614284
                             ...                       
7075      262670271815966144641543642228426261415962414
7076     4644221558262614334915436159615962846143901262
7077    30699159643091543661426246442228417952626141596
7078     6141596154361983464262422377582621596940614284
7079     1543661442215964644761262192315966141899262284
Name: input_str, Length: 7080, dtype: object

In [153]:
x_train['input_str'] = x_train['input_str'].str.lstrip('0') 

In [154]:
x_train

,input_str
0,1543646427181596614280026242231322841596262614
1,46415963695154364222626141104715962624761614284
2,15436262159659996144224641358061426240511596284
3,1543642246412386142621596895615962843510614262
4,46418994221543620690159661426247612621596614284
...,...
7075,262670271815966144641543642228426261415962414
7076,4644221558262614334915436159615962846143901262
7077,30699159643091543661426246442228417952626141596
7078,6141596154361983464262422377582621596940614284


In [164]:
def extract_substrings(s):
    return [s[i:j] for i in range(len(s)) for j in range(i + 1, len(s) + 1)]

# Create a list to hold all substrings
all_substrings = []

# Iterate over each string in the DataFrame
for string in x_train['input_str']:
    substrings = extract_substrings(string)
    all_substrings.extend(substrings)

# Create a new DataFrame for the substrings
sub_df = pd.DataFrame(all_substrings, columns=['substring'])

# Display the DataFrame of substrings
sub_df

,substring
0,1
1,15
2,154
3,1543
4,15436
...,...
7804211,28
7804212,284
7804213,8
7804214,84


In [179]:
aggregated_df = sub_df.groupby('substring').size().reset_index(name='Frequency').sort_values(by='Frequency', ascending=False)


In [181]:
aggregated_df = aggregated_df.drop(aggregated_df[aggregated_df['Frequency'] > 7080].index)
aggregated_df = aggregated_df.drop(aggregated_df[aggregated_df['Frequency'] < 2].index)
aggregated_df

,substring,Frequency
426962,15436,7080
3649873,626,6631
4122640,7,6067
856458,215,5907
2234202,426,5723
...,...,...
4452946,96154364644205,2
187869,1235614154,2
4404425,9422614464159,2
229737,141007015962,2


In [182]:
# Define the emoji you are looking for
emoji_to_find = '😛'

# Find all rows in y_train where the emoji is present
rows_with_emoji = y_train[y_train['input_emoticon'].str.contains(emoji_to_find)]

# Display the rows
print(rows_with_emoji)

     input_emoticon
0     😛🛐😻😑😣🙠🙯🚼😒🙼😑🙯😣
1     🛐😑😪😛🚼🙯😣🚅😑🙯😹😣🙼
2     😛🙯😑🚡😣🚼🛐🙲😣🙯🛑😑🙼
3     😛🚼🛐🙐😣🙯😑🙪😑🙼🛆😣🙯
4     🛐🚟🚼😛🙋😑😣🙯😹🙯😑😣🙼
...             ...
7075  🙯😺😻😑😣🛐😛🚼🙼🙯😣😑🚠
7076  🛐🚼😅🙯😣🙹😛😑😑🙼😣🚍🙯
7077  🛜😑🙒😛😣🙯🛐🚼🙼🙬🙯😣😑
7078  😣😑😛🛋🛐🙯🚼🚪🙯😑🛀😣🙼
7079  😛😣🚼😑🛐😹🙯🚭😑😣🚟🙯🙼

[7080 rows x 1 columns]


In [183]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [185]:
import numpy as np

# Create a similarity matrix
n = len(aggregated_df)
similarity_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i + 1, n):
        sim = similarity(sub_df.iloc[i]['substring'], sub_df.iloc[j]['substring'])
        similarity_matrix[i, j] = sim
        similarity_matrix[j, i] = sim  # Symmetric matrix

# Convert to a distance matrix (1 - similarity)
distance_matrix = 1 - similarity_matrix


KeyboardInterrupt: 

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Perform clustering
n_clusters = 3  # You can adjust this number
clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='average')
labels = clustering.fit_predict(distance_matrix)

# Add cluster labels to the substrings DataFrame
aggregated_df['cluster'] = labels
print(aggregated_df)

In [ ]:
for cluster in range(n_clusters):
    print(f"\nCluster {cluster}:")
    print(sub_df[sub_df['cluster'] == cluster]['substring'].tolist())
